In [7]:
import sys
sys.path.append('../')
import torch 
import torch.nn as nn
from torchvision import datasets,models,transforms
import pandas as pd 
import os
import glob
import numpy as np
from src import config 

In [5]:
train_df = pd.read_csv(os.path.join('..',config.TRAINING_DATA,'train.csv'))
train_df.head()

,filename,label
0,0-with-mask.jpg,with_mask
1,1-with-mask.jpg,with_mask
2,10-with-mask.jpg,with_mask
3,100-with-mask.jpg,with_mask
4,101-with-mask.jpg,with_mask


In [6]:
test_df = pd.read_csv(os.path.join('..',config.TESTING_DATA,'test.csv'))
test_df.head()

,filename,label
0,augmented_image_227.jpg,with_mask
1,augmented_image_228.jpg,with_mask
2,augmented_image_23.jpg,with_mask
3,augmented_image_230.jpg,with_mask
4,augmented_image_232.jpg,with_mask


In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]), 
    'test' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [35]:
data_dir = '../data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}

In [36]:
image_datasets['train']

Dataset ImageFolder
    Number of datapoints: 1096
    Root location: ../data/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [37]:
image_datasets['test']

Dataset ImageFolder
    Number of datapoints: 276
    Root location: ../data/test
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [38]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                             batch_size=16, 
                                             shuffle=True, 
                                             num_workers=4) 
               for x in ['train', 'test']}

In [39]:
class_names = image_datasets['train'].classes

In [40]:
class_names

['with_mask', 'without_mask']

In [41]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [42]:
device

device(type='cpu')

In [43]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

In [44]:
dataset_sizes

{'train': 1096, 'test': 276}

In [46]:
model_ft = models.resnet101(pretrained=True)

num_frts = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_frts, len(class_names))

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
optimizer_ft = torch.optim.Adagrad(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)